In [1]:
import sqlite3
import requests
from entities.city import City
from typing import List
import pandas as pd
from sqlite3 import Connection

conn = sqlite3.connect('db/weather.db')

c = conn.cursor()

def get_cities() -> pd.DataFrame:
    with conn:
        c.execute('SELECT * FROM city')
        data = c.fetchall()
        cols = ['id', 'name', 'latitude', 'longitude', 'voivodeship', 'elevation']
        return pd.DataFrame(data=data, columns=cols)
# Another way:
def get_cities2() -> pd.DataFrame:
    return pd.read_sql('SELECT * FROM city', conn)

df = get_cities()
df2 = get_cities2()

def get_api() -> pd.DataFrame:
    with conn:
        c.execute('SELECT * FROM api')
        data = c.fetchall()
        cols = ['id', 'name', 'latitude', 'longitude', 'voivodeship', 'elevation']
        return pd.DataFrame(data=data, columns=cols)

In [10]:
import sqlite3
import pandas as pd
import requests
import json
from typing import Optional
import time
from datetime import datetime
from sqlite3 import Connection
from entities.api_call import APICall
import json
import time

### Call setup:
URL = 'https://api.open-meteo.com/v1/forecast'
HOURLY_PARAMS = 'temperature_2m,relative_humidity_2m,precipitation_probability,precipitation,weather_code,pressure_msl,surface_pressure,wind_speed_10m,cloud_cover'
CURRENT_PARAMS = 'temperature_2m,relative_humidity_2m,apparent_temperature,is_day,precipitation,weather_code,cloud_cover,pressure_msl,surface_pressure,wind_speed_10m'
timezone = 'Europe/Berlin'

### Connection:
conn = sqlite3.connect('db/weather.db')
cursor = conn.cursor()

### Cities Fetch:
def fetch_cities(conn: Connection) -> pd.DataFrame:
    return pd.read_sql(sql='SELECT * FROM city', con=conn)

### Storing API Call
def store_api_call(conn: Connection, api_call: APICall) -> Optional[int]:
    with conn:
        cursor.execute("""INSERT OR IGNORE INTO api_call
                    (call_timestamp, status, cities_fetched)
                    VALUES (:call_timestamp, :status, :cities_fetched)""", 
                    {
                        'call_timestamp': api_call.call_timestamp,
                        'status': api_call.status,
                        'cities_fetched': api_call.cities_fetched,
                        
                    })
    return cursor.lastrowid

def update_api_call(conn: Connection, api_call: APICall, api_call_id: int | None):
    with conn:
        cursor.execute("""UPDATE api_call
            SET status = :status,
                cities_fetched = :cities_fetched
            WHERE id=:api_call_id
            """,
            {
                'status': api_call.status,
                'cities_fetched': api_call.cities_fetched,
                'api_call_id': api_call_id 
            })

def fetch_store_weather(conn: Connection, cities_df: pd.DataFrame) -> int | None:
    
    #Create API call record
    api_call = APICall(
        call_timestamp=datetime.now(),
        status='in_progress',
        cities_fetched=0
    )

    call_id = store_api_call(conn=conn, api_call=api_call)
    
    current_records = []
    forecast_records = []
    
    #Fetch weather for each city
    for _, city in cities_df.iterrows():
        
        response = requests.get(
            url = URL,
            params={
                'latitude': city['latitude'],
                'longitude': city['longitude'],
                'hourly': HOURLY_PARAMS,
                'current': CURRENT_PARAMS,
                'timezone' : timezone
            }
        )
        
        data = response.json()
        
        #Parse current weather
        current = data['current']
        current_records.append({
            'api_call_id': call_id,
            'city_id': city['id'],
            'time': datetime.fromisoformat(current['time']),
            'temperature_2m': current.get('temperature_2m'),
            'relative_humidity_2m': current.get('relative_humidity_2m'),
            'apparent_temperature': current.get('apparent_temperature'),
            'is_day': current.get('is_day'),
            'precipitation' : current.get('precipitation'),
            'weather_code' : current.get('weather_code'),
            'cloud_cover' : current.get('cloud_cover'),
            'pressure_msl' : current.get('pressure_msl'),
            'surface_pressure' : current.get('surface_pressure'),
            'wind_speed_10m' : current.get('wind_speed_10m')
            			
        })
        
        #Parse hourly forecasts
        hourly = data['hourly']
        for i, time_str in enumerate(hourly['time']):
            forecast_records.append({
                'api_call_id': call_id,
                'city_id': city['id'],
                'time': datetime.fromisoformat(time_str),
                'temperature_2m': hourly['temperature_2m'][i],
                'relative_humidity_2m': hourly['relative_humidity_2m'][i],
                'precipitation_probability': hourly['precipitation_probability'][i],
                'precipitation': hourly['precipitation'][i],
                'weather_code': hourly['weather_code'][i],
                'cloud_cover': hourly['cloud_cover'][i],
                'pressure_msl': hourly['pressure_msl'][i],
                'surface_pressure': hourly['surface_pressure'][i],
                'wind_speed_10m': hourly['wind_speed_10m'][i]
            })

    #Bulk insert (fast!) ----- NEED TO FINISH THIS ONE NEXT
    with conn:
        cursor.executemany('''
            INSERT OR REPLACE INTO weather_current
            (api_call_id,city_id,time,temperature_2m,relative_humidity_2m,
            apparent_temperature,is_day,precipitation,weather_code,
            cloud_cover,pressure_msl,surface_pressure,wind_speed_10m)
            VALUES (:api_call_id,:city_id,:time,:temperature_2m,:relative_humidity_2m,
            :apparent_temperature,:is_day,:precipitation,:weather_code,
            :cloud_cover,:pressure_msl,:surface_pressure,:wind_speed_10m)
            ''', current_records)
        
        cursor.executemany('''
            INSERT OR REPLACE INTO weather_forecast
                (api_call_id,city_id,time,temperature_2m,relative_humidity_2m, 
                precipitation_probability,precipitation,weather_code,
                cloud_cover,pressure_msl,surface_pressure,wind_speed_10m)

            VALUES (:api_call_id,:city_id,:time,:temperature_2m,:relative_humidity_2m,
                :precipitation_probability,:precipitation,:weather_code,
                :cloud_cover,:pressure_msl,:surface_pressure,:wind_speed_10m)
                ''', forecast_records)
    
    #Update API call status
    api_call.status = 'completed'  # type: ignore
    api_call.cities_fetched = len(cities_df)  # type: ignore
    update_api_call(conn=conn, api_call=api_call, api_call_id=call_id)
    
    time.sleep(0.5)
    
    print(f"API Call {call_id}: Stored {len(current_records)} current + {len(forecast_records)} forecasts")
    return call_id

# Usage

cities = fetch_cities(conn=conn)
api_call = fetch_store_weather(conn=conn, cities_df=cities)
print(f"API CALL {api_call} was succesful!")
conn.close()

C:\Users\jakob\AppData\Local\Temp\ipykernel_23564\3234839024.py:30: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""INSERT OR IGNORE INTO api_call
C:\Users\jakob\AppData\Local\Temp\ipykernel_23564\3234839024.py:123: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany('''
C:\Users\jakob\AppData\Local\Temp\ipykernel_23564\3234839024.py:133: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany('''


API Call 5: Stored 16 current + 2688 forecasts
API CALL 5 was succesful!
